# Przewidywanie bankructwa polskich przedsiębiorstw

Celem projektu jest zbudowanie modelu klasyfikacyjnego, który na podstawie wskaźników finansowych przewidzi upadłość firmy.


### Importowanie bibliotek i konfiguracja środowiska


In [ ]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import wandb
from scipy.io import arff
from dotenv import load_dotenv

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    recall_score,
    precision_recall_curve,
    f1_score,
)
from xgboost import XGBClassifier

from attributes import attributes_pl

# Ustawienia wyświetlania
pd.set_option("display.max_columns", None)
plt.style.use("seaborn-v0_8-whitegrid")

# Ustawienia kolorów
COLOR_HEALTHY = "#1f77b4"
COLOR_BANKRUPT = "#d62728"

YEAR_TO_ANALYZE = 3  # od 1 do 5
print(f"Analizowany rok: {YEAR_TO_ANALYZE}")

## Część 1: Inżynieria danych i model bazowy

1.  **Wczytanie i unifikacja danych:** Wczytanie plików `.arff` i unifikacja.
2.  **Eksploracyjna analiza danych (EDA):** Zrozumienie danych, braków i korelacji.
3.  **Przetworzenie i podział danych:** Czyszczenie, podział, imputacja i skalowanie danych.
4.  **Analiza głównych składowych (PCA):** Redukcja wymiarowości.
5.  **Model bazowy (baseline):** Budowa prostego modelu odniesienia (regresja logistyczna).


### Wczytanie i unifikacja danych

Wczytanie danych z pliku `.arff`. Dane zawierają wskaźniki finansowe (Attr1 - Attr64) oraz etykietę klasy (`class`), gdzie:

- `0` - firma zdrowa
- `1` - bankrut

Następnie dane są dzielone zbiór na treningowy i testowy (proporcja 80/20).


In [ ]:
file_path = f"data/{YEAR_TO_ANALYZE}year.arff"

try:
    data, meta = arff.loadarff(file_path)
    df = pd.DataFrame(data)

    df["class"] = df["class"].astype(int)

    print(f"Wczytano dane dla roku {YEAR_TO_ANALYZE}")
    print(f"Wymiary: {df.shape[0]} wierszy, {df.shape[1]} kolumn")

    display(df.head(5))

except FileNotFoundError:
    print(f"BŁĄD: Nie odnaleziono pliku: {file_path}")

### Eksploracyjna analiza danych (EDA)

1.  **Analiza braków danych:** Sprawdzenie, które wskaźniki finansowe są najczęściej niekompletne.
2.  **Rozkład klas:** Weryfikacja, jak bardzo niezbalansowany jest zbiór (stosunek firm zdrowych do bankrutów).
3.  **Korelacje:** Szukanie cech, które mają najsilniejszy związek (dodatni lub ujemny) z bankructwem.


In [ ]:
# Ustawienia kolorów
BINARY_PALETTE = [COLOR_HEALTHY, COLOR_BANKRUPT]

cmap_diverging = mcolors.LinearSegmentedColormap.from_list(
    "CustomRdBu", [COLOR_HEALTHY, "white", COLOR_BANKRUPT]
)

# 1. ANALIZA BRAKÓW DANYCH
missing = df.isnull().sum() / len(df) * 100
missing = missing[missing > 0].sort_values(ascending=False)

plt.figure(figsize=(12, 6))

if not missing.empty:
    top_missing = missing.head(20)

    ax = sns.barplot(
        x=top_missing.index,
        y=top_missing.values,
        hue=top_missing.index,
        legend=False,
        palette=sns.light_palette(COLOR_BANKRUPT, n_colors=25, reverse=True),
    )

    for container in ax.containers:
        ax.bar_label(container, fmt="%.1f%%", padding=3)

    plt.title("Cechy z największymi brakami danych")
    plt.xlabel("Cecha")
    plt.ylabel("% braków")
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.xticks(rotation=45)

    max_missing = top_missing.max()
    limit = max_missing * 1.15 if max_missing > 0 else 10

    plt.ylim(0, limit)

    plt.show()

    print("LEGENDA")
    for attr in top_missing.index[:20]:
        print(f"{attr}: {attributes_pl.get(attr, 'Brak opisu')}")
else:
    print("Brak pustych wartości")

# 2. ROZKŁAD KLAS
plt.figure(figsize=(6, 5))

ax = sns.countplot(
    x="class", data=df, hue="class", legend=False, palette=BINARY_PALETTE
)

total = len(df)

for container in ax.containers:
    labels = [
        f"{int(v.get_height())} ({v.get_height() / total * 100:.2f}%)"
        for v in container
    ]
    ax.bar_label(container, labels=labels, label_type="edge", padding=3)

plt.title("Liczba firm według statusu")
plt.xlabel("Status firmy")
plt.ylabel("Liczba firm")
plt.xticks([0, 1], ["0 (Zdrowa)", "1 (Bankrut)"])

max_height = max([p.get_height() for p in ax.patches])

plt.ylim(0, max_height * 1.2)

plt.show()

# 3. KORELACJE
correlations = df.corr()["class"].sort_values()
top_corr = correlations.abs().sort_values(ascending=False).head(11)
top_corr_names = top_corr.index.tolist()

if "class" in top_corr_names:
    top_corr_names.remove("class")

plt.figure(figsize=(10, 6))

corr_values = correlations[top_corr_names].values
norm = plt.Normalize(corr_values.min(), corr_values.max())
colors = cmap_diverging(norm(corr_values))

ax = sns.barplot(
    x=corr_values,
    y=top_corr_names,
    hue=corr_values,
    legend=False,
    palette=cmap_diverging,
)

for container in ax.containers:
    ax.bar_label(container, fmt="%.2f", padding=3)

plt.title("Cechy najsilniej skorelowane z bankructwem")
plt.xlabel("Współczynnik korelacji (Pearson)")
plt.grid(axis="x", linestyle="--", alpha=0.5)

min_val = corr_values.min()
max_val = corr_values.max()
padding = max(abs(min_val), abs(max_val)) * 0.2

plt.xlim(min_val - padding, max_val + padding)

plt.show()

print("LEGENDA")
for attr in top_corr_names:
    print(f"{attr}: {attributes_pl.get(attr, 'Brak opisu')}")

### Przetworzenie i podział danych

Na podstawie analizy braków danych zdecydowano usunąć cechę `Attr37`, która posiada zbyt wiele pustych wartości, by je bezpiecznie uzupełniać.

Następnie:

1.  **Podział (Train/Test):** Dane dzielone są w proporcji 80/20 z zachowaniem proporcji klas (`stratify`).
2.  **Pipeline:**
    - **Imputacja:** Braki uzupełniane są medianą.
    - **Skalowanie:** Dane są standaryzowane (`StandardScaler`), co jest wymagane dla PCA i regresji logistycznej.


In [ ]:
# 1. PRZYGOTOWANIE DANYCH
# Usunięcie cechy Attr37
if "Attr37" in df.columns:
    X = df.drop(["class", "Attr37"], axis=1)
    print("Usunięto kolumnę Attr37")
else:
    X = df.drop("class", axis=1)
    print("Kolumna Attr37 nie istnieje")

y = df["class"]

feature_names_final = X.columns.tolist()
print(f"Liczba cech: {X.shape[1]}")

# 2. PODZIAŁ DANYCH
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Rozmiar zbioru treningowego: {X_train.shape}")
print(f"Rozmiar zbioru testowego:  {X_test.shape}")
print(f"Liczba bankrutów w zbiorze testowym: {y_test.sum()} (na {len(y_test)} firm)")

# 3. PIPELINE PRZETWARZANIA DANYCH
preprocessor = Pipeline(
    [
        (
            "imputer",
            SimpleImputer(strategy="median"),
        ),  # mediana jest odporna na wartości odstające
        ("scaler", StandardScaler()),  # średnia=0, odchylenie=1
    ]
)

# 4. DOPASOWANIE I TRANSFORMACJA DANYCH
# Pipeline jest dopasowywany do zbioru treningowego
X_train_scaled = preprocessor.fit_transform(X_train)
# Zbiór testowy jest transformowany na podstawie parametrów wyuczonych na zbiorze treningowym
X_test_scaled = preprocessor.transform(X_test)

# Zamiana na DataFrame (dla wygody operowania nazwami kolumn)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names_final)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names_final)

### Analiza głównych składowych (PCA)

Dane mają 63 wymiary, co utrudnia wizualizację. Zastosowano PCA, aby sprawdzić, czy bankruci tworzą oddzielne skupisko.


In [ ]:
# Ustawienia kolorów
palette_dict = {"Zdrowa": COLOR_HEALTHY, "Bankrut": COLOR_BANKRUPT}

# 1. URUCHOMIENIE PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled)

df_pca = pd.DataFrame(data=X_pca, columns=["PC1", "PC2"])
df_pca["class"] = y_train.values
df_pca["Legenda"] = df_pca["class"].map({0: "Zdrowa", 1: "Bankrut"})

# 2. LICZBA WARIANCJI WYJAŚNIONEJ
evr = pca.explained_variance_ratio_
print(f"Wariancja wyjaśniona przez PC1: {evr[0]:.2%}")
print(f"Wariancja wyjaśniona przez PC2: {evr[1]:.2%}")
print(f"Suma informacji na wykresie: {sum(evr):.2%}")

# 3. WIZUALIZACJA PCA
plt.figure(figsize=(10, 7))
sns.scatterplot(
    x="PC1",
    y="PC2",
    hue="Legenda",
    data=df_pca,
    palette=palette_dict,
    alpha=0.8,
    edgecolor=None,
)
plt.title("PCA: Przestrzeń 2D cech")
plt.xlabel(f"PC1 ({evr[0]:.2%} wariancji)")
plt.ylabel(f"PC2 ({evr[1]:.2%} wariancji)")
plt.legend(title="Status firmy")
plt.grid(True, alpha=0.3)
plt.show()

# 4. WYBÓR LICZBY KOMPONENTÓW PCA
pca_full = PCA().fit(X_train_scaled)

plt.figure(figsize=(10, 5))
plt.plot(
    np.cumsum(pca_full.explained_variance_ratio_),
    marker=".",
    linestyle="-",
    color=COLOR_HEALTHY,
)
plt.xlabel("Liczba komponentów")
plt.ylabel("Skumulowana wariancja wyjaśniona")
plt.title("Liczba komponentów PCA a wyjaśniona wariancja")
plt.axhline(y=0.95, color=COLOR_BANKRUPT, linestyle="--", label="Próg 95%")
plt.legend()
plt.grid(True, alpha=0.5)
plt.show()

### Model bazowy (baseline): Regresja logistyczna

Jako punkt odniesienia wytrenowana została regresja logistyczna.
Używany jest parametr `class_weight='balanced'`, aby zmusić model do zwracania uwagi na mniejszą klasę bankrutów (w przeciwnym razie model mógłby ignorować bankrutów i wciąż mieć wysoką ogólną dokładność).


In [ ]:
# 1. TRENING
log_reg = LogisticRegression(class_weight="balanced", max_iter=2000, random_state=42)

print("Trening modelu bazowego...")
log_reg.fit(X_train_scaled, y_train)

# 2. PREDYKCJA
y_pred_base = log_reg.predict(X_test_scaled)
y_proba_base = log_reg.predict_proba(X_test_scaled)[:, 1]

# 3. WYNIKI
roc_auc_base = roc_auc_score(y_test, y_proba_base)
recall_base = recall_score(y_test, y_pred_base)
f1_score_base = f1_score(y_test, y_pred_base)

print("WYNIKI")
print(f"ROC AUC: {roc_auc_base:.4f}")
print(f"Recall: {recall_base:.4f}")
print(f"F1 Score: {f1_score_base:.4f}")
print("Confusion matrix:")
print(classification_report(y_test, y_pred_base))

# 4. WIZUALIZACJE (macierz pomyłek i krzywa ROC)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Macierz Pomyłek
cmap_cm = sns.light_palette(COLOR_HEALTHY, as_cmap=True)

cm = confusion_matrix(y_test, y_pred_base)
sns.heatmap(cm, annot=True, fmt="d", cmap=cmap_cm, cbar=False, ax=ax[0])

ax[0].set_title("Macierz pomyłek")
ax[0].set_xlabel("Przewidziana klasa")
ax[0].set_ylabel("Prawdziwa klasa")
ax[0].set_xticklabels(["Zdrowa", "Bankrut"])
ax[0].set_yticklabels(["Zdrowa", "Bankrut"])

# Krzywa ROC
fpr, tpr, thresholds = roc_curve(y_test, y_proba_base)

ax[1].plot(
    fpr,
    tpr,
    label=f"Regresja logistyczna (ROC AUC = {roc_auc_base:.2f})",
    color=COLOR_HEALTHY,
    linewidth=3,
)
ax[1].fill_between(fpr, tpr, color=COLOR_HEALTHY, alpha=0.1)
ax[1].plot([0, 1], [0, 1], color="gray", linestyle="--", alpha=0.7)
ax[1].set_xlabel("False Positive (fałszywe alarmy)")
ax[1].set_ylabel("True Positive (wykrywalność)")
ax[1].set_title("Krzywa ROC")
ax[1].legend(loc="lower right")
ax[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Część 2: Zaawansowane modelowanie

1.  **Random Forest:** Model odporny na overfitting.
2.  **XGBoost:** Obecny standard w konkursach ML.
3.  **Threshold tuning:** Manipulacja progiem decyzyjnym, aby zmaksymalizować wykrywalność bankrutów (recall).
4.  **WandB:** Śledzenie eksperymentów w chmurze.


In [ ]:
# Import słownika
try:
    from attributes import attributes_pl
except ImportError:
    attributes_pl = {}

# 1. ZAŁADOWANIE KONFIGURACJI
load_dotenv()

# Pobranie klucza API
api_key = os.getenv("WANDB_API_KEY")

if not api_key:
    print("UWAGA: Nie znaleziono WANDB_API_KEY")

try:
    print("Logowanie do WandB...")
    wandb.login(key=api_key)
    print("Zalogowano pomyślnie")
except Exception as e:
    print(f"BŁĄD: Nieudana próba logowania do WandB: {e}")

### Eksperyment 1: Las losowe (random forest)

Algorytm, który buduje wiele drzew decyzyjnych i uśrednia ich wyniki.

- **Strategia na niezbalansowanie:** Użyto `class_weight='balanced'`, co automatycznie zwiększa kary za błędy na klasie mniejszościowej (bankrutach).
- **Feature importance:** Model ten pozwala łatwo ocenić, które wskaźniki finansowe są kluczowe dla predykcji.


In [ ]:
# 1. DEFINICJA WARIANTÓW PARAMETRÓW
param_grid = {
    "n_estimators": [100, 500, 1000],  # liczba drzew
    "max_depth": [10, 20, None],  # głębokość drzew
    "min_samples_leaf": [1, 2, 4],  # minimalna liczba próbek w liściu
}

fixed_params = {
    "model_type": "Random Forest Tuned",
    "class_weight": "balanced",
    "random_state": 42,
    "n_jobs": 1,
    "year": YEAR_TO_ANALYZE,
}

# 2. SZUKANIE NAJLEPSZEGO MODELU
CV_FOLDS = 3  # liczba podziałów walidacji krzyżowej

total_combinations = np.prod([len(v) for v in param_grid.values()])
total_trainings = total_combinations * CV_FOLDS
print(
    f"Szukanie najlepszego modelu ({total_combinations} kombinacji, {total_trainings} treningów)..."
)
start_search = time.time()

rf_temp = RandomForestClassifier(class_weight="balanced", random_state=42, n_jobs=1)

grid_search = GridSearchCV(
    estimator=rf_temp,
    param_grid=param_grid,
    scoring="roc_auc",  # szukanie najwyższego ROC AUC
    cv=CV_FOLDS,  # walidacja krzyżowa
    n_jobs=-1,
)

grid_search.fit(X_train_scaled, y_train)

print(f"Poszukiwania zakończono w {time.time() - start_search:.2f} s")

# 3. WYBÓR ZWYCIĘZCY
best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("NAJLEPSZE HIPERPARAMETRY:")
print(f"Liczba drzew (n_estimators): {best_params['n_estimators']}")
print(f"Głębokość drzew (max_depth): {best_params['max_depth']}")
print(
    f"minimalna liczba próbek w liściu (min_samples_leaf): {best_params['min_samples_leaf']}"
)

# 4. KONFIGURACJA RUNU
run_config_best = fixed_params.copy()
run_config_best.update(best_params)

# 5. INICJALIZACJA RUNU
run = wandb.init(
    project="polish-bankruptcy-prediction",
    config=run_config_best,
    name=f"RF_TUNED_Year{YEAR_TO_ANALYZE}",
)

# 6. EWALUACJA ZWYCIĘZCY
rf_model = best_rf_model

y_pred_rf = rf_model.predict(X_test_scaled)
y_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_score_rf = f1_score(y_test, y_pred_rf)

print("WYNIKI")
print(f"ROC AUC: {roc_auc_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1 Score: {f1_score_rf:.4f}")
print("Confusion matrix:")
print(classification_report(y_test, y_pred_rf))

# 7. WIZUALIZACJA WAŻNOŚCI CECH
importances = rf_model.feature_importances_
feature_imp_df = (
    pd.DataFrame({"Feature": X_train_scaled.columns, "Importance": importances})
    .sort_values(by="Importance", ascending=False)
    .head(10)
)

feature_imp_df["Opis"] = (
    feature_imp_df["Feature"].map(attributes_pl).fillna(feature_imp_df["Feature"])
)

plt.figure(figsize=(10, 6))
custom_palette = sns.light_palette(COLOR_HEALTHY, n_colors=10, reverse=True)
ax = sns.barplot(
    x="Importance",
    y="Opis",
    hue="Opis",
    data=feature_imp_df,
    palette=custom_palette,
    legend=False,
)
for container in ax.containers:
    ax.bar_label(container, fmt="%.4f", padding=3)

plt.title("Najważniejsze cechy")
plt.xlabel("Ważność (wskaźnik Giniego)")
plt.ylabel("")
plt.xlim(0, feature_imp_df["Importance"].max() * 1.15)
plt.show()

# 8. LOGOWANIE WYNIKÓW DO WANDB
wandb.log(
    {
        "roc_auc": roc_auc_rf,
        "recall": recall_rf,
        "f1_score": f1_score_rf,
        "confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=y_test.values,
            preds=y_pred_rf,
            class_names=["Zdrowa", "Bankrut"],
        ),
        "roc_curve": wandb.plot.roc_curve(
            y_test.values,
            rf_model.predict_proba(X_test_scaled),
            labels=["Zdrowa", "Bankrut"],
        ),
    }
)

run.finish()

### Analiza progu decyzyjnego (threshold tuning)

Większość modeli domyślnie klasyfikuje firmę jako bankruta, jeśli prawdopodobieństwo wynosi $> 50\%$. W przypadku danych niezbalansowanych to podejście często zawodzi – model jest zbyt _ostrożny_.

Poniżej analizowana jest **krzywa precyzja-czułość**, która przedstawia dylemat:

- Czy wykrywać wszystkich bankrutów (**wysoka czułość**), ale mieć dużo fałszywych alarmów?
- Czy mieć pewność, że oznaczone przedsiębiorstwo na pewno zbankrutuje (**wysoka precyzja**), ale wielu zostanie przeoczonych?


In [ ]:
# 1. DEFINICJA NOWEGO PROGU
NEW_THRESHOLD = 0.2
print(f"Nowy próg decyzyjny: {NEW_THRESHOLD}")

# 2. DOSTOSOWANIE PREDYKCJI
y_pred_adjusted = (y_proba_rf >= NEW_THRESHOLD).astype(int)

# 3. SPRAWDZENIE NOWYCH WYNIKÓW
print("WYNIKI")
print("Confusion matrix:")
print(classification_report(y_test, y_pred_adjusted))

# 4. WIZUALIZACJA DYLEMATU
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_rf)

plt.figure(figsize=(10, 6))
plt.plot(
    thresholds,
    precisions[:-1],
    label="Precyzja",
    color=COLOR_HEALTHY,
    linewidth=2,
    linestyle="--"
)
plt.plot(thresholds, recalls[:-1], label="Czułość", color=COLOR_BANKRUPT, linewidth=2)
plt.xlabel("Próg decyzyjny (threshold)")
plt.ylabel("Wartość metryki")
plt.title("Dylemat: precyzja a czułość")
plt.legend()
plt.grid(True, alpha=0.3)
plt.axvline(x=NEW_THRESHOLD, color="gray", linestyle=":", label=f"Próg {NEW_THRESHOLD}")

plt.show()

### Eksperyment 2: XGBoost (eXtreme Gradient Boosting)

Aby poradzić sobie z małą liczbą bankrutów, zastosowano parametr `scale_pos_weight`. Mówi on modelowi, że **błąd na bankrucie jest X razy bardziej kosztowny** niż błąd na zdrowej firmie.


In [ ]:
# 1. OBLICZENIE WAGI KLASY MNIEJSZOŚCIOWEJ
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Wyliczona waga dla klasy bankrutów: {scale_pos_weight:.2f}")

# 2. DEFINICJA WARIANTÓW PARAMETRÓW
param_grid = {
    "n_estimators": [100, 500, 1000],  # liczba drzew
    "max_depth": [4, 6, 8, 10],  # głębokość drzew
    "learning_rate": [0.01, 0.05, 0.1],  # szybkość uczenia
}

fixed_params = {
    "model_type": "XGBoost Tuned",
    "scale_pos_weight": scale_pos_weight,
    "random_state": 42,
    "n_jobs": 1,
    "eval_metric": "auc",
    "year": YEAR_TO_ANALYZE,
}

# 3. SZUKANIE NAJLEPSZEGO MODELU
CV_FOLDS = 3 # liczba podziałów walidacji krzyżowej

total_combinations = np.prod([len(v) for v in param_grid.values()])
total_trainings = total_combinations * CV_FOLDS
print(
    f"Szukanie najlepszego modelu ({total_combinations} kombinacji, {total_trainings} treningów)..."
)
start_search = time.time()

xgb_temp = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=1,
    eval_metric="auc",
)

grid_search = GridSearchCV(
    estimator=xgb_temp,
    param_grid=param_grid,
    scoring="roc_auc",  # szukanie najwyższego ROC AUC
    cv=CV_FOLDS,  # walidacja krzyżowa
    n_jobs=-1,
)

grid_search.fit(X_train_scaled, y_train)

print(f"Poszukiwania zakończono w {time.time() - start_search:.2f} s")

# 4. WYBÓR ZWYCIĘZCY
best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("NAJLEPSZE HIPERPARAMETRY:")
print(f"Liczba drzew (n_estimators): {best_params['n_estimators']}")
print(f"Głębokość drzew (max_depth): {best_params['max_depth']}")
print(f"Szybkość uczenia (learning_rate): {best_params['learning_rate']}")

# 5. KONFIGURACJA RUNU
run_config_best = fixed_params.copy()
run_config_best.update(best_params)

# 6. INICJALIZACJA RUNU
run = wandb.init(
    project="polish-bankruptcy-prediction",
    config=run_config_best,
    name=f"XGB_TUNED_Year{YEAR_TO_ANALYZE}",
)

# 7. EWALUACJA
xgb_model = best_xgb_model

y_pred_xgb = xgb_model.predict(X_test_scaled)
y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_score_xgb = f1_score(y_test, y_pred_xgb)

print("WYNIKI")
print(f"ROC AUC: {roc_auc_xgb:.4f}")
print(f"Recall: {recall_xgb:.4f}")
print(f"F1 Score: {f1_score_xgb:.4f}")
print("Confusion matrix:")
print(classification_report(y_test, y_pred_xgb))

# 8. WIZUALIZACJA WAŻNOŚCI CECH
importances = xgb_model.feature_importances_
feature_imp_df = (
    pd.DataFrame({"Feature": X_train_scaled.columns, "Importance": importances})
    .sort_values(by="Importance", ascending=False)
    .head(10)
)

feature_imp_df["Opis"] = (
    feature_imp_df["Feature"].map(attributes_pl).fillna(feature_imp_df["Feature"])
)

plt.figure(figsize=(10, 6))
custom_palette = sns.light_palette(COLOR_HEALTHY, n_colors=10, reverse=True)
ax = sns.barplot(
    x="Importance",
    y="Opis",
    hue="Opis",
    data=feature_imp_df,
    palette=custom_palette,
    legend=False,
)
for container in ax.containers:
    ax.bar_label(container, fmt="%.4f", padding=3)

plt.title("Najważniejsze cechy")
plt.xlabel("Ważność (gain)")
plt.ylabel("")
plt.xlim(0, feature_imp_df["Importance"].max() * 1.15)
plt.show()

# 9. LOGOWANIE WYNIKÓW DO WANDB
wandb.log(
    {
        "roc_auc": roc_auc_xgb,
        "recall": recall_xgb,
        "f1_score": f1_score_xgb,
        "confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=y_test.values,
            preds=y_pred_xgb,
            class_names=["Zdrowa", "Bankrut"],
        ),
        "roc_curve": wandb.plot.roc_curve(
            y_test.values,
            xgb_model.predict_proba(X_test_scaled),
            labels=["Zdrowa", "Bankrut"],
        ),
        "feature_importance": wandb.plot.bar(
            wandb.Table(dataframe=feature_imp_df[["Feature", "Importance"]]),
            "Feature",
            "Importance",
            title="Najważniejsze cechy",
        ),
    }
)

run.finish()

### Finalna optymalizacja: Dostrojenie progu pod biznes

W kontekście bankowym/inwestycyjnym koszt przeoczenia bankructwa (utrata kapitału) jest drastycznie wyższy niż koszt sprawdzenia fałszywego alarmu.

Dlatego w tym kroku **nie ufa się domyślnemu progowi**. Ustawiono cel biznesowy na **wykrycie co najmniej 85% bankrutów** (recall $\approx$ 0,85) i sprawdzono, jaki próg prawdopodobieństwa pozwoli to osiągnąć.


In [ ]:
# 1. PRZYGOTOWANIE KONFIGURACJI
opt_config = run_config_best.copy()
opt_config["threshold_tuning"] = True
opt_config["model_stage"] = "optimization"

# 2. INICJALIZACJA RUNU
run = wandb.init(
    project="polish-bankruptcy-prediction",
    name=f"XGB_Optimization_Year{YEAR_TO_ANALYZE}",
    config=opt_config,
)

# 3. PREDYKCJA POCZĄTKOWA
y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]  # z poprzedniego kroku

# 4. OPTYMALIZACJA PROGU DECYZYJNEGO
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_xgb)

TARGET_RECALL = 0.85  # cel
optimal_idx = np.argmin(np.abs(recalls - TARGET_RECALL))
optimal_threshold = thresholds[optimal_idx]

print(f"Cel: Recall ~ {TARGET_RECALL * 100}%")
print(f"Optymalny próg: {optimal_threshold:.4f}")

# 5. PREDYKCJA Z NOWYM PROGIEM
y_pred_opt = (y_proba_xgb >= optimal_threshold).astype(int)

# 6. EWALUACJA FINALNA
final_roc_auc = roc_auc_score(y_test, y_proba_xgb)
final_recall = recall_score(y_test, y_pred_opt)
final_f1_score = f1_score(y_test, y_pred_opt)

print("WYNIKI")
print(f"ROC AUC: {final_roc_auc:.4f}")
print(f"Recall: {final_recall:.4f}")
print(f"F1 Score: {final_f1_score:.4f}")
print("Confusion matrix:")
print(classification_report(y_test, y_pred_opt))

# 7. WIZUALIZACJA MACIERZY POMYŁEK
plt.figure(figsize=(6, 5))

cmap_opt = sns.light_palette(COLOR_BANKRUPT, as_cmap=True)

cm = confusion_matrix(y_test, y_pred_opt)

sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap=cmap_opt,
    cbar=False,
    xticklabels=["Zdrowa", "Bankrut"],
    yticklabels=["Zdrowa", "Bankrut"],
)

plt.title(f"Macierz pomyłek (próg {optimal_threshold:.4f})")
plt.xlabel("Przewidziana klasa")
plt.ylabel("Prawdziwa klasa")
plt.show()

# 8. LOGOWANIE WYNIKÓW DO WANDB
wandb.log(
    {
        "optimal_threshold": optimal_threshold,
        "final_auc": final_roc_auc,
        "final_recall": final_recall,
        "final_f1": final_f1_score,
        "final_confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=y_test.values,
            preds=y_pred_opt,
            class_names=["Zdrowa", "Bankrut"],
        ),
    }
)

run.finish()